In [2]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
import time
from dateutil import relativedelta
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By  
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl
import os
#import dotenv
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

[WDM] - Downloading: 100%|██████████| 1.75M/1.75M [00:01<00:00, 1.79MB/s]


'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,event_list_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

In [32]:
def albemarle_county_pc(url):
    url_test = verify_url(url)
    messages=[]
    empty_messages = []
    if url_test==True:
        driver.get(url)
        time.sleep(5)
        table_rows = driver.find_elements(By.CSS_SELECTOR, "tr")
        #page we're looking at only has upcoming meetings, so no need to check the date
        meeting_links = driver.find_elements(By.CSS_SELECTOR,"a[href*=Calendar")
        meeting_times = driver.find_elements(By.CSS_SELECTOR,"td[class*=event_datetime")
        meeting_links_href=[]
        for item in meeting_links:
            meeting_links_href.append(item.get_attribute("href"))
        meeting_times_text=[]
        for item in meeting_times:
            meeting_times_text.append(item.text)
        main_window = driver.window_handles[0]
        meeting_titles = []
        for i in range(0,len(meeting_times)):
            meeting_titles.append(meeting_links[i].text + " " + meeting_times[i].text)
        for i in range(0,len(meeting_links)):
            driver.get(meeting_links_href[i])
            time.sleep(2)
            main_window=driver.window_handles[0]
            try:
                agenda_link = driver.find_element(By.CSS_SELECTOR,"a[href*=showpublisheddocument")
                agenda_link.click()
                time.sleep(2)
                agenda_window=driver.window_handles[1]
                driver.switch_to.window(agenda_window)
                agenda_content = driver.find_elements(By.CSS_SELECTOR,"div[class*=textLayer")
                agenda_search=search_agenda_for_keywords(agenda_content)
                driver.close()
                driver.switch_to.window(main_window)
                driver.get(url)
                time.sleep(2)
                if agenda_search != []:
                    messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Albemarle County Planning Commission in " + meeting_titles[i])
            except:
                empty_messages.append("No agenda available for "+ meeting_titles[i])
    if url_test != True:
        messages.append(url_test)
    return messages

In [38]:
driver.get("https://www.albemarle.org/government/community-development/boards-and-commissions/planning-commission/-toggle-next30days")

In [39]:
table_rows = driver.find_elements(By.CSS_SELECTOR, "tr")

In [46]:
table_rows[0].text

'EVENT DATE/TIME'

In [43]:
meeting_links = [item.find_element(By.CSS_SELECTOR,"a[href*=Calendar") for item in table_rows]
for item in table_rows

NoSuchElementException: Message: Unable to locate element: a[href*=Calendar
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16
